<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
from tensorflow import keras
from keras.optimizers import SGD
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv('Customer.csv')

In [2]:
def cat_encode(X):
    
    X = X.copy()

    # import labelencoder
    from sklearn.preprocessing import LabelEncoder
    
    le = LabelEncoder()

    #Categorical boolean mask
    categorical_feature_mask = X.dtypes==object
    categorical_cols = X.columns[categorical_feature_mask].tolist()

    X[categorical_cols] = X[categorical_cols].apply(lambda col: le.fit_transform(col))
    
    return X

In [3]:
df = cat_encode(df)
target = 'Churn'
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

X = df[features]
y = df[target]

#Identify target in our df and set the predicted value to the mode
y_true = df['Churn']
majority = df['Churn'].mode()[0]

#Create a list of predictions for the length of our df
y_pred = [majority] * len(y_true)

#Use accuracy_score to check our baseline accuracy
from sklearn.metrics import accuracy_score
print("Baseline Accuracy:", accuracy_score(y_true, y_pred))

Baseline Accuracy: 0.7346301292063041


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 19), (1409, 19), (5634,), (1409,))

In [5]:
batch_size = 20
num_classes = 2
epochs = 10

opt = SGD(lr=0.1, momentum=0.9)

In [6]:
model = Sequential()
model.add(Dense(5, activation='relu', input_shape=(19,)))
model.add(Dense(3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy', 
    optimizer=opt, 
    metrics=['accuracy'],
)

history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    epochs=epochs,
    batch_size=batch_size,
    verbose=0)

In [7]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.740241289138794


In [8]:
def create_model():  
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(19,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

# create model
model = KerasClassifier(
    build_fn=create_model, 
    verbose=0, 
    batch_size=2, 
)

# define the grid search parameters
param_grid = {
    'epochs': [5],
}

In [9]:
# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7376596689224243 using {'epochs': 5}
Means: 0.7376596689224243, Stdev: 0.015438246081261318 with: {'epochs': 5}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?

In [10]:
def create_model_beta(depth, drop, activation):  
    model = Sequential()
    model.add(Dense(depth, activation=activation, input_shape=(19,)))
    model.add(Dropout(drop))
    model.add(Dense(depth, activation=activation))
    model.add(Dropout(drop))
    model.add(Dense(depth, activation=activation))
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

In [11]:
model = KerasClassifier(
    verbose=0,
    build_fn=create_model_beta,
    batch_size=16,
    epochs=100,
    drop=0.15,
    activation='linear',
)

param_distributions = {
#     'batch_size': range(12, 24, 2),
#     'epochs': range(80, 150, 5),
#     'drop': [0.1, 0.15, 0.2, 0.25, 0.3],
#     'activation': ['tanh', 'relu', 'linear', 'softmax', 'softplus', 'softsign', 'sigmoid', 'hard_sigmoid'],
#     'activation': ['relu', 'linear', 'softplus'],
    'depth': range(100, 400, 100),
}

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    cv=3,
    n_iter=3,
    n_jobs=8,
    random_state=42,
)

In [12]:
# Fit RandomizedSearchCV Model
search_result = search.fit(X_train, y_train)

# Report Results
print(f"Best: {search_result.best_score_} using {search_result.best_params_}")
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7964146335919698 using {'depth': 300}
Means: 0.7941072185834249, Stdev: 0.005590361026398337 with: {'depth': 100}
Means: 0.7774227857589722, Stdev: 0.012697935333049132 with: {'depth': 200}
Means: 0.7964146335919698, Stdev: 0.0025101539057128196 with: {'depth': 300}


### Take 2

In [13]:
def create_model(n_nodes, drop, activation, input_shape, output_size, n_layers):
    model = Sequential()
    model.add(Dense(n_nodes, activation=activation, input_shape=input_shape))
    model.add(Dropout(drop))

    for _ in range(n_layers):
        model.add(Dense(n_nodes, activation=activation))
        model.add(Dropout(drop))

    if output_size == 1:
        model.add(Dense(output_size, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    else:
        model.add(Dense(output_size, activation='softmax'))
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model

It can take a while to search all these params at once!

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

classifier = KerasClassifier(
    build_fn=create_model,
    input_shape=X_train.columns.shape,
    output_size=1,
#     batch_size=12,
#     epochs=40,
#     drop=0.1,
#     activation='softplus',
#     n_nodes=64,
#     n_layers=4,
)

param_distributions = {
    'batch_size': range(2, 24, 2),
    'epochs': range(20, 200, 20),
    'drop': [0.1, 0.15, 0.2, 0.25, 0.3],
    'activation': ['tanh', 'relu', 'linear', 'softmax', 'softplus', 'softsign', 'sigmoid', 'hard_sigmoid'],
    'n_nodes': range(32, 161, 32),
    'n_layers': range(2, 6),
}

search = RandomizedSearchCV(
    estimator=classifier,
    param_distributions=param_distributions,
    cv=3,
    n_iter=100,
    n_jobs=8,
    random_state=42,
)

search_result = search.fit(X_train, y_train)

print(f"Best: {search_result.best_score_} using {search_result.best_params_}")
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']

Epoch 1/40
529/529 [==============================] - 1s 1ms/step - loss: 5.0408 - accuracy: 0.6221
Epoch 2/40
529/529 [==============================] - 1s 1ms/step - loss: 0.8551 - accuracy: 0.6671
Epoch 3/40
529/529 [==============================] - 1s 1ms/step - loss: 0.6722 - accuracy: 0.7072: 0s - loss: 0.6723 - accuracy: 0.
Epoch 4/40
529/529 [==============================] - 1s 1ms/step - loss: 0.6102 - accuracy: 0.7250
Epoch 5/40
529/529 [==============================] - 1s 1ms/step - loss: 0.5957 - accuracy: 0.7300
Epoch 6/40
529/529 [==============================] - 1s 1ms/step - loss: 0.5842 - accuracy: 0.7330
Epoch 7/40
529/529 [==============================] - 1s 1ms/step - loss: 0.5778 - accuracy: 0.7319
Epoch 8/40
529/529 [==============================] - 1s 1ms/step - loss: 0.5758 - accuracy: 0.7359
Epoch 9/40
529/529 [==============================] - 1s 1ms/step - loss: 0.5739 - accuracy: 0.7367
Epoch 10/40
529/529 [==============================] - 1s 1ms/step